Get going on sorting people into leagues for curling
KL 8/21/2016

This version starts with a random league and puts a random subset of people into each league

In [76]:
if False:
    %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [77]:
import pandas as pd
import numpy as np
import random
import math
import csv
import os.path

#used this to step into the function and debug it, also need line with Tracer()() 
from IPython.core.debugger import Tracer 

In [78]:
#read in the file with the information on people's priorities
#goal will be to use the version that comes out of the website
fName = 'TestPriority4.csv'
regInfo=pd.read_csv(fName)

In [79]:
##OK, now that I have the data in Python, add some columns that I will need
#first, how many numbers so I need?
nPeople = len(regInfo.index)
regInfo['randomNumCol'] = np.random.choice(range(nPeople), nPeople,replace = False)
##so now, moving back to the list of people, sort the list by the random number
#...still need to randomize people bc might have case where I can't  
regSorted = regInfo.sort_values(by = 'randomNumCol', ascending=True)
regSorted = regSorted.set_index(['randomNumCol'])

#copy so I can have an output tracking what was done, make after I put in league managers
regTracking = regSorted.copy(deep=True)

Define some functions up top

In [80]:
def addOne(leagueDict,oneLeague,oneName):
    if len(leagueDict[oneLeague])==0: #nothing there yet
        leagueDict[oneLeague] = oneName
    else:     #need to append
        leagueDict[oneLeague] = leagueDict[oneLeague].append(oneName)
    return leagueDict

In [81]:
def addManager(leagueDictionary,r,manager): #here r is the index in...
    useRow = r.index[0]
    oneName = regSorted.loc[useRow,['lastname', 'firstname']] 
    oneName = oneName.to_frame().transpose() #need this format to make nice output
    leagueDictionary[manager] = oneName
    regSorted.loc[useRow,manager]=np.nan
    regTracking.loc[useRow,manager]='yes'
    return leagueDictionary

get started on the actual lists

In [82]:
#need the list of open leagues...
openLeagues = list(regInfo)[5:12]
openLeagues.remove('League_4_Men')
randLeagues = list(openLeagues) #make a copy of the list
random.shuffle(randLeagues) #does this in place, so don't make a new variable
randLeagues

['League_2_Monday_Open',
 'League_Tuesday_Open',
 'League_5_Wednesday_Open',
 'League_6_Thursday_Open',
 'League_1_Sunday_Open',
 'League_3_Monday_Night_Open']

In [83]:
#setup the dictionary to hold the people information (start with openLeagues only)
forDictionary = openLeagues
#forDictionary.extend(otherLeagues)
leagueDict_open = {}
for name in forDictionary:
    leagueDict_open[name] = pd.DataFrame()

In [84]:
#put the league managers into their league and set their the choice for that league to NaN
manager = 'League_1_Sunday_Open'
r = regSorted.loc[(regSorted['lastname']=='Fragaszy') & (regSorted['firstname']=='Bill')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_2_Monday_Open'
r = regSorted.loc[(regSorted['lastname']=='Brown') & (regSorted['firstname']=='Peter')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_3_Monday_Night_Open'
r = regSorted.loc[(regSorted['lastname']=='DiMassa') & (regSorted['firstname']=='Diane')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_Tuesday_Open'
r = regSorted.loc[(regSorted['lastname']=='Thomson') & (regSorted['firstname']=='Kenny')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_5_Wednesday_Open'
r = regSorted.loc[(regSorted['lastname']=='Murphy') & (regSorted['firstname']=='Katie')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_6_Thursday_Open'
r = regSorted.loc[(regSorted['lastname']=='Gallagher') & (regSorted['firstname']=='Bill')]
leagueDict_open = addManager(leagueDict_open,r,manager)


In [85]:
#set the limit for each league:
sevenTeams = 5 #change to 28/27 (? should league manager goes in first)
fourteenTeams = 55

d = {'League_1_Sunday_Open': sevenTeams,
     'League_2_Monday_Open': sevenTeams,
     'League_3_Monday_Night_Open': fourteenTeams,
     'League_Tuesday_Open': sevenTeams,
     'League_5_Wednesday_Open': sevenTeams,
     'League_6_Thursday_Open': sevenTeams,}

nLimit = pd.DataFrame({'nLimit' : d})
del d

In [86]:
issueNames = ['notInFirst','notInSecond','notInThird','noSecondChoice','noThirdChoice']

#setup the dictionary to hold the issues
trackIssues = {}
for name in issueNames:
    trackIssues[name] = pd.DataFrame()

del issueNames

In [87]:
pd.options.mode.chained_assignment = None  # default='warn'

In [88]:
#change to track issues in a new data frame
for oneLeague in randLeagues:    
    #how many spaces are left in this league?
    nRemaining = nLimit.loc[oneLeague][0] - len(leagueDict_open[oneLeague])
    if nRemaining > 0:      
        #find list of people interested in that league
        subset = regSorted.loc[regSorted[oneLeague]==1]
        subset.reset_index(inplace = True)
        nPeople = len(subset.index)
        subset.loc[:,('subsetRandomNumCol')] = np.random.choice(range(nPeople),nPeople, replace = False)

        for idx in range(0,nPeople): #remember nPeople in this version is only subset for a league
            oneName = subset.loc[idx,['lastname', 'firstname']] 
            oneName = oneName.to_frame().transpose() #need this format to make nice output

            #does this person want another league?
            rt = regTracking.loc[idx,openLeagues]
            cHave = len(rt[rt=='yes'])
            del rt
            
            if cHave < regTracking.loc[idx,'numberLeagues']: #person wants more 
                if subset.loc[idx,'subsetRandomNumCol'] < (nRemaining): #random # < number of places remaining
                    leagueDict_open = addOne(leagueDict_open,oneLeague,oneName)                
                    regSorted.loc[idx,oneLeague] = np.nan
                    regTracking.loc[idx,oneLeague] = 'yes'

                else:
                    #no space in first choice league:
                    trackIssues = addOne(trackIssues,'notInFirst',oneName)
                    #set their first choice to 'no' and move onto second choices
                    newI = subset.loc[idx,'randomNumCol'] ###remember: this is the index into regSorted and regTracking
                    regTracking.loc[newI,oneLeague] = 'no'
                    regSorted.loc[newI,oneLeague] = np.nan

                    #use the idx into subset to go back into regTracking
                    newPriority = regSorted.loc[newI,openLeagues].dropna().min()

                    if math.isnan(newPriority) == False: #have another league listed
                        league2 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                        #first need to check if the league has space
                        if len(leagueDict_open[league2]) < (nLimit.loc[league2,'nLimit']):
                            leagueDict_open = addOne(leagueDict_open,league2,oneName)
                            regTracking.loc[newI,league2] = 'yes'
                            regSorted.loc[newI,league2] = np.nan
                        else:
                            #no space in second choice league
                            trackIssues = addOne(trackIssues,'notInSecond',oneName)
                            #set their first choice to 'no' and move onto second choices
                            regTracking.loc[newI,league2] = 'no' 
                            regSorted.loc[newI,league2] = np.nan

                            #use the idx into subset to go back into regTracking
                            priority3 = regSorted.loc[newI,openLeagues].dropna().min()

                            if math.isnan(priority3) == False: #have another league listed
                                league3 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                                #first need to check if the league has space
                                if len(leagueDict_open[league3]) < (nLimit.loc[league3,'nLimit']):
                                    leagueDict_open = addOne(leagueDict_open,league3,oneName)
                                    regTracking.loc[newI,league3] = 'yes'
                                    regSorted.loc[newI,league3] = np.nan
                                else:
                                    print oneName.values.ravel()[1],oneName.values.ravel()[0],': no space in third choice league'
                                    trackIssues = addOne(trackIssues,'notInThird',oneName)
                                    regTracking.loc[newI,league3] = 'no'
                            else: 
                                #no other league listed
                                trackIssues = addOne(trackIssues,'noThirdChoice',oneName)
                    elif math.isnan(newPriority):
                        #print oneName.values.ravel()[1],oneName.values.ravel()[0], ': did not list a second league'
                        trackIssues = addOne(trackIssues,'noSecondChoice',oneName)
        del subset
    elif nRemaining == 0:
        print oneLeague, 'no more space in league'

League_5_Wednesday_Open no more space in league


In [89]:
#make a new dictionary for other leagues...concat with openLeagues when done
# do doubles and competitive league by hand...
otherLeagues = ['League_4_Men', 'League_7_Ladies' ,'League_Scrod']

leagueDict_other = {}
for name in otherLeagues:
    leagueDict_other[name] = pd.DataFrame()

In [90]:
#put league managers in for other
manager = 'League_4_Men'
r = regSorted.loc[(regSorted['lastname']=='Pijanowski') & (regSorted['firstname']=='Brian')]
leagueDict_other = addManager(leagueDict_other,r,manager)

manager = 'League_7_Ladies'
r = regSorted.loc[(regSorted['lastname']=='Galligan') & (regSorted['firstname']=='Natalie')]
leagueDict_other = addManager(leagueDict_other,r,manager)

manager = 'League_Scrod'
r = regSorted.loc[(regSorted['lastname']=='Pilotte') & (regSorted['firstname']=='Courtney Lee')]
leagueDict_other = addManager(leagueDict_other,r,manager)

In [91]:
#only have to go through once
for idx in range(0,nPeople):
    for oL in otherLeagues:    
        oneName = regSorted.loc[idx,['lastname', 'firstname']] 
        oneName = oneName.to_frame().transpose() #need this format to make nice output
        if regSorted.loc[idx,oL]=='Yes': #careful with trailing spaces
            leagueDict_other = addOne(leagueDict_other,oL,oneName)
            #now that I have added the person to a league, change their priority to NaN
            regTracking.loc[idx,oL] = 'yes'
            regSorted.loc[idx,oL] = np.nan

In [92]:
#combine the two dictionaries
leagueDict_all = leagueDict_open.copy()
leagueDict_all.update(leagueDict_other)

#combine the list of names
allLeagues = list(openLeagues)
allLeagues.extend(otherLeagues)

In [93]:
#finally, export the results (CAREFUL...will delete existing files)
for fn in allLeagues:
    fName = '%s.csv' % fn
    #check if the file exists...delete if it does
    if os.path.isfile(fName):
        os.remove(fName)
        
    leagueDict_all[fn].to_csv(fName)

In [94]:
#count the number of people with issues

In [95]:
trackIssues

{'noSecondChoice':         lastname  firstname
 0         Belfit  Gabrielle
 7       Spilhaus     Connie
 9         Becker       Meg 
 0           Lino     Meghan
 14        Ryczek    Carolyn
 0   Austin-Leary       Shay
 9         barnes      chuck
 13     Snellings       Emma
 20         Bruce   Gabriele
 21        Colgan  Katherine
 22        Brandt  Alexandra
 24       Yaroch      Jeanie
 25       Maloney      Kathy
 26        Harlow   Kathleen, 'noThirdChoice':        lastname firstname
 7          Mojo     Steve
 9     Weinstein      Joel
 14  Lousararian   Richard
 15       Bailey    James , 'notInFirst':         lastname  firstname
 0         Belfit  Gabrielle
 7       Spilhaus     Connie
 8       Benedict      Karin
 9         Becker       Meg 
 14        Lemcke     Russel
 15        Palmer      David
 17       Perella       Anne
 20       Gerlach       Will
 0           Lino     Meghan
 7      Woodworth      James
 8          Balas     Martha
 9       Mitchell       Pete
 13 

In [96]:
# leave here bc I keep having to look this up
#df.loc[df['column_name'] == some_value]
regTracking.loc[regTracking['lastname']=='Bundy']

,Register Date,Event Name,firstname,lastname,numberLeagues,League_1_Sunday_Open,League_2_Monday_Open,League_3_Monday_Night_Open,League_Tuesday_Open,League_4_Men,...,League_6_Thursday_Open,League_7_Ladies,League_Friday_Doubles,League_Friday_Doubles2,League_Friday_Doubles3,Competitive_League,Competitive_League_Part2,League_Scrod,League_Day_Curling,League_Sub
randomNumCol,,,,,,,,,,,,,,,,,,,,,
27,1/5/2016 22:08,2015-2016 Second Half League Registration,Randie,Bundy,0,1,NaN,NaN,NaN,NaN,...,NaN,NaN,No,NaN,NaN,No,NaN,NaN,No,"Sunday Night Open, Monday Night Open, Scrod Le..."
